In [1]:
import random
from scipy.stats import norm
import numpy as np
import math
import pandas as pd
import time
import os

c:\programdata\miniconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\programdata\miniconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
c:\programdata\miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


# 0. Preprocessing 

### 0.1 Distributions (hobbies, heights, birth month, favorite color) 

In [2]:
# ------ hobbies -------
hobbies = ['music', 'travel', 'reading/writing', 'food', 'socializing', 'playing sports', 'health and fitness', 'pets', 'technology/computers', 'gardening', 'video games', 'watching sports', 'photography', 'dancing', 'arts and crafts', 'other']
hobby_probabilities = [0.5, 0.44, 0.42, 0.38, 0.36, 0.34, 0.34, 0.32, 0.28, 0.27, 0.14, 0.12, 0.09, 0.2]
# values extracted from Figure 1 in "Partysimulation_Analysis.docx"

# ------ heights -------
men = [197]*63 + [187]*128 + [182]*239 + [177]*261 + [172]*192 + [167]*90 + [162]*23 + [157]*3 + [152]
women = [187]*2 + [182]*18 + [177]*69 + [172]*176 + [167]*291 + [162]*270 + [157]*127 + [152]*40 + [143]*6
# values extracted from Figure 2 in "Partysimulation_Analysis.docx"
mens_mean, mens_var  = norm.fit(men)
womens_mean, womens_var  = norm.fit(women)

# ------ birth month -------
months = range(1,13)
month_weights = [1]*12

# ------ favourite colour -------
colours = ['purple', 'blue', 'green', 'yellow', 'orange', 'red', 'pink']
colour_weights_men = [12, 42, 25, 5, 7, 8, 1]
colour_weights_women = [27, 29, 19, 6, 3, 9, 7]
# values extracted from Figure 3 in "Partysimulation_Analysis.docx"

# ------ sock colour -------
sock_colours = ['black', 'blue', 'white', 'brown', 'grey', 'green', 'red', 'multi-colour']
sock_colour_weights = [40, 14, 12, 5, 12, 5, 6, 6]
# values extracted from Figure 4 in "Partysimulation_Analysis.docx"


### 0.2 Agent's rule learning functions

In [3]:
def get_persons_data(person):
    vector = []
    
    # gender
    vector.append(int(person['gender']=='male'))
    
    # hobbies
    for hobby in hobbies:
        if hobby in person['hobbies']:
            vector.append(1)
        else: 
            vector.append(0)
            
    # height
    vector.append((person['height'][0]-100)/100)
    
    # birth month
    vector.append((person['birth month']/12))
    
    # favourite colour
    for colour in colours:
        vector.append(int(person['favourite colour']==colour)) 
    
    # sock colour
    for colour in sock_colours:
        vector.append(int(person['sock colour']==colour))  
    return vector


def get_personal_information_vector(timestep, tables, table_nb, place_nb):
    X_values = [int(timestep%2==0), int(timestep%3==0)]
    X_values += get_persons_data(tables[table_nb][place_nb])
    for relative_person_nb in range(len(tables[table_nb])):
        if relative_person_nb%2 == 0:
            this_persons_place_nb = place_nb + int(relative_person_nb/2) + 1
            this_persons_place_nb = this_persons_place_nb%len(tables[table_nb])
            X_values += get_persons_data(tables[table_nb][this_persons_place_nb])
        else:
            this_persons_place_nb = place_nb - int(relative_person_nb/2)
            this_persons_place_nb = this_persons_place_nb%len(tables[table_nb])
            X_values += get_persons_data(tables[table_nb][this_persons_place_nb])
            
    # fill up the remaining X_values with zeros    
    X_values += [0]*34 *(20 - len(tables[table_nb]))
            
    y_value = tables[table_nb][place_nb]['happy']
    return (X_values, y_value)




# 1. Functions

### 1.1 Initialize simulation

In [4]:
def initialize_simulation(number_of_tables, number_of_women, number_of_men):
    tables = []
    for table_nb in range(number_of_tables):
        tables.append([])
    for i in range(number_of_women):

        woman = {'gender':'female', 'happy':True}
        # 1.1 hobby
        womans_hobbies = []
        for hobby, probability in zip(hobbies, hobby_probabilities):
            if random.random() < probability:
                womans_hobbies += [hobby]
        woman['hobbies'] = womans_hobbies

        # 1.2 height
        woman['height'] = np.random.normal(womens_mean, math.sqrt(womens_var), 1)

        # 1.3 birth month
        woman['birth month'] = random.choices(months, weights=month_weights, k=1)[0]

        # 1.4 favourite colour
        woman['favourite colour'] = random.choices(colours, weights=colour_weights_women, k=1)[0]

        # 1.5 sock colour
        woman['sock colour'] = random.choices(sock_colours, weights=sock_colour_weights, k=1)[0]


        table_nb = random.choice(range(number_of_tables))
        tables[table_nb].append(woman)

    for i in range(number_of_men):

        man = {'gender':'male', 'happy':True}
        # 1.1 hobby
        mans_hobbies = []
        for hobby, probability in zip(hobbies, hobby_probabilities):
            if random.random() < probability:
                womans_hobbies += [hobby]
        man['hobbies'] = mans_hobbies

        # 1.2 height
        man['height'] = np.random.normal(mens_mean, math.sqrt(mens_var), 1)

        # 1.3 birth month
        man['birth month'] = random.choices(months, weights=month_weights, k=1)[0]

        # 1.4 favourite colour
        man['favourite colour'] = random.choices(colours, weights=colour_weights_men, k=1)[0]

        # 1.5 sock colour
        man['sock colour'] = random.choices(sock_colours, weights=sock_colour_weights, k=1)[0]

        table_nb = random.choice(range(number_of_tables))
        tables[table_nb].append(man)

    return tables

### 1.2 Run Simulation

In [5]:
def run_simulation(run_name, nb_of_timesteps, tables, check_happiness_function, log, get_agent_learning_data):
    
    all_logs = []
    all_agent_learning_data = []
    for timestep in range(nb_of_timesteps):
        log_dict = {}
        

        # PART 1 - Check happiness
        start_time = time.time()
        for table_nb, table in enumerate(tables):
            for persons_place, inspected_person in enumerate(table):
                happy = check_happiness_function(timestep, persons_place, inspected_person, table)
                tables[table_nb][persons_place]['happy'] = happy
        happiness_checking_duration = time.time() - start_time
        
        
        # Logging 1 ---------------
        total_nb_of_unhappy = 0
        for table_nb, table in enumerate(tables):
            nb_of_unhappy = 0
            for persons_place, inspected_person in enumerate(table):
                if not happy:
                    nb_of_unhappy += 1  
                    total_nb_of_unhappy += 1
            log_dict['nb_of_unhappy_table'+ str(table_nb)] = nb_of_unhappy
        
        if get_agent_learning_data:
            for table_nb, table in enumerate(tables):
                for persons_place, inspected_person in enumerate(table):
                    if get_agent_learning_data:
                        (X_values, y_value) = get_personal_information_vector(timestep, tables, table_nb, persons_place)
                        X_column_names = ['X_' + str(index) for index in range(len(X_values))]
                        agent_learning_row = {col_name: value for col_name, value in zip(X_column_names, X_values)}
                        agent_learning_row['y'] = y_value
                        all_agent_learning_data.append(agent_learning_row)
        # ----------------------------
            

            
        # PART 2 - Reseat the unhappy
        seat_finding_duration = 0
        next_timestep_tables = []
        for table_nb in range(number_of_tables):
            next_timestep_tables.append([])

        for table_nb, table in enumerate(tables):
            other_tables = list(range(number_of_tables))
            other_tables.remove(table_nb)
            destination_from_last_mover = -1
            for person in table:
                if person['happy']:
                    next_timestep_tables[table_nb].append(person)
                else:
                    person['happy'] = True
                    
                    seatfinding_start = time.time() 
                    if check_happiness_function == check_happiness_g4:
                        if destination_from_last_mover >= 0:
                            destination = destination_from_last_mover
                            destination_from_last_mover = -1
                        else:
                            destination = random.choice(other_tables)
                            destination_from_last_mover = destination
                    else:
                        destination = random.choice(other_tables)
                    seat_finding_duration += time.time() - seatfinding_start
                    next_timestep_tables[destination].append(person)

        
        
        
        # Logging 2 ----------------
        # get table distribution for logging
        if log:
            for table_nb, table in enumerate(tables):
                female = 0
                male = 0
                for person in table:
                    if person['gender'] == 'female':
                        female += 1
                    else:
                        male += 1
                log_dict['number_of_women_table' + str(table_nb)] = female
                log_dict['number_of_men_table' + str(table_nb)] = male
            log_dict['happiness_checking_duration'] = happiness_checking_duration + seat_finding_duration

            all_logs.append(log_dict)
        # ---------------------------- 
            
            
            
        # PART 3 - Step
        tables = next_timestep_tables  
        
    
    
    # Logging 3 ----------------
    if log:    
        logs_df = pd.DataFrame(all_logs, index=range(len(all_logs)))
        logs_df.to_csv('./logs/'+ run_name + '.csv', index=False)
    if get_agent_learning_data:
        agent_learning_data_df = pd.DataFrame(all_agent_learning_data, index=range(len(all_agent_learning_data)))
        agent_learning_data_df.to_csv('./agent learning data/'+ run_name + '.csv', index=False)
    # ---------------------------- 


### 1.3 Check happiness functions

In [6]:
# ==============================================
# Default Game (unhappy if fewer than 50% of the other people have the same gender)
# ==============================================
def check_happiness_default(timestep, persons_place, inspected_person, table):
    same_gender = 0
    other_gender = 0
    for person in table:
        if person['gender'] == inspected_person['gender']:
            same_gender += 1
        else:
            other_gender += 1
    if len(table)>1:
        happy = ((same_gender-1)/(same_gender+other_gender-1) > 0.5)
    else: 
        happy = True
    return happy
    
    
    
# ==============================================    
# Game 1
# ==============================================
def check_happiness_g1(timestep, persons_place, inspected_person, table):
    if timestep%2==0:
        return False
    else: 
        return True

    
# ==============================================        
# Game 2
# ==============================================
def check_happiness_g2(timestep, persons_place, inspected_person, table):
    happy = False
    
    #hobbies
    if timestep%4==0: 
        for place, person in enumerate(table):
            if place != persons_place:
                hobbies_in_common = set.intersection(set(person['hobbies']), set(inspected_person['hobbies']))
                if len(hobbies_in_common) >0:
                    happy = True
    
    # heights (happy if height difference < 4cm)  
    elif timestep%4==1: 
        for place, person in enumerate(table):
            if place != persons_place:
                if abs(person['height'] - inspected_person['height']) < 4:
                    happy = True
    
    # birth month
    elif timestep%4==2:
        for place, person in enumerate(table):
            if place != persons_place:
                if person['birth month'] == inspected_person['birth month']:
                    happy = True
        
    # favourite colour
    else:
        for place, person in enumerate(table):
            if place != persons_place:
                if person['favourite colour'] == inspected_person['favourite colour']:
                    happy = True
    
    return happy
                

# ==============================================    
# Game 3
# ==============================================
def check_happiness_g3(timestep, persons_place, inspected_person, table):
    happy = True
    
    left_place = (persons_place - 1) % len(table)
    right_place = (persons_place + 1) % len(table)
    if left_place != persons_place and table[left_place]['sock colour'] == 'white':
        happy = False
    if right_place != persons_place and table[right_place]['sock colour'] == 'white':
        happy = False
    
    return happy


# ==============================================
# Game 4
# ==============================================
def check_happiness_g4(timestep, persons_place, inspected_person, table):
    same_gender = 0
    other_gender = 0
    for person in table:
        if person['gender'] == inspected_person['gender']:
            same_gender += 1
        else:
            other_gender += 1
            
    happy = True
    if len(table)>1:
        if (other_gender)/(same_gender+other_gender) > 0.5:
            if inspected_person['birth month'] in [1, 2, 3, 4, 5, 6]:
                happy = False

    return happy
    

# ==============================================    
# Game 5
# ==============================================
def check_happiness_g5(timestep, persons_place, inspected_person, table):
    return random.random() < 0.5


# ==============================================        
# Additional Game: Only hobbies
# ==============================================
def check_happiness_g6(timestep, persons_place, inspected_person, table):
    happy = False
    
    for place, person in enumerate(table):
        if place != persons_place:
            hobbies_in_common = set.intersection(set(person['hobbies']), set(inspected_person['hobbies']))
            if len(hobbies_in_common) >0:
                happy = True
    return happy


# ==============================================        
# Additional Game: Only heights
# ==============================================
def check_happiness_g7(timestep, persons_place, inspected_person, table):
    happy = False
    
    for place, person in enumerate(table):
        if place != persons_place:
            if abs(person['height'] - inspected_person['height']) < 4:
                happy = True
   
    return happy



# ==============================================        
# Additional Game: Only birth month
# ==============================================
def check_happiness_g8(timestep, persons_place, inspected_person, table):
    happy = False    

    for place, person in enumerate(table):
        if place != persons_place:
            if person['birth month'] == inspected_person['birth month']:
                happy = True

    return happy



# ==============================================        
# Additional Game: Only favourite colour
# ==============================================
def check_happiness_g9(timestep, persons_place, inspected_person, table):
    happy = False  
 
    for place, person in enumerate(table):
        if place != persons_place:
            if person['favourite colour'] == inspected_person['favourite colour']:
                happy = True
    
    return happy

# 2. Run simulations!

In [7]:
# # !!!!!!!!!!!!!!!!! delete all existing logs !!!!!!!!!!!!!!!

folder = './logs/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    os.unlink(file_path)

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [8]:
# # !!!!!!!!!!!!!!!!! delete all existing agent learning data !!!!!!!!!!!!!!!

# folder = './agent learning data/'
# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
# os.unlink(file_path)

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [10]:
number_of_tables = 5
number_of_women = 11
number_of_men = 9
log = True
get_agent_learning_data = False
nb_of_timesteps = 100

# default game - 1000 runs
#for run_nb in range(1000):
#    run_name = 'default_game__run' + str(run_nb)
#    print(run_name)
#    tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
#    run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_default, log, get_agent_learning_data)

# game 1 - 1000 runs
for run_nb in range(1000):
   run_name = 'game1__run' + str(run_nb)
   print(run_name)
   tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
   run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g1, log, get_agent_learning_data)
    
# game 2 - 1000 runs
for run_nb in range(1000):
    run_name = 'game2__run' + str(run_nb)
    print(run_name)
    tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
    run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g2, log, get_agent_learning_data)
    
# game 3 - 1000 runs
for run_nb in range(1000):
    run_name = 'game3__run' + str(run_nb)
    print(run_name)
    tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
    run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g3, log, get_agent_learning_data)
    
# game 4 - 1000 runs
for run_nb in range(1000):
    run_name = 'game4__run' + str(run_nb)
    print(run_name)
    tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
    run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g4, log, get_agent_learning_data)
    
# game 5 - 1000 runs
for run_nb in range(1000):
    run_name = 'game5__run' + str(run_nb)
    
    tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
    run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g5, log, get_agent_learning_data)
    
    

# # additional game: only hobbies - 1000 runs
# for run_nb in range(1000):
#     run_name = 'addition_only_hobbies__run' + str(run_nb)
#     print(run_name)
#     tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
#     run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g6, log, get_agent_learning_data)

# # additional game: only heights - 1000 runs
# for run_nb in range(1000):
#     run_name = 'addition_only_heights__run' + str(run_nb)
#     print(run_name)
#     tables = initialize_simulation(number_of_tables, number_of_women, number_of_men)
#     run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g7, log, get_agent_learning_data)
    
# # additional game: only birth month - 1000 runs
# for run_nb in range(1000):
#     run_name = 'addition_only_birth_month__run' + str(run_nb)
#     print(run_name)
#     tables = initialize_simulation(number_of_tables, number_of_women, number_of_men, get_agent_learning_data)
#     run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g8, log)
    
# # additional game: only favourite colour - 1000 runs
# for run_nb in range(1000):
#     run_name = 'addition_only_favourite_colour__run' + str(run_nb)
#     print(run_name)
#     tables = initialize_simulation(number_of_tables, number_of_women, number_of_men, get_agent_learning_data)
#     run_simulation(run_name, nb_of_timesteps, tables,  check_happiness_g9, log)
    

game1__run0
game1__run1
game1__run2
game1__run3
game1__run4
game1__run5
game1__run6
game1__run7
game1__run8
game1__run9
game1__run10
game1__run11
game1__run12
game1__run13
game1__run14
game1__run15
game1__run16
game1__run17
game1__run18
game1__run19
game1__run20
game1__run21
game1__run22
game1__run23
game1__run24
game1__run25
game1__run26
game1__run27
game1__run28
game1__run29
game1__run30
game1__run31
game1__run32
game1__run33
game1__run34
game1__run35
game1__run36
game1__run37
game1__run38
game1__run39
game1__run40
game1__run41
game1__run42
game1__run43
game1__run44
game1__run45
game1__run46
game1__run47
game1__run48
game1__run49
game1__run50
game1__run51
game1__run52
game1__run53
game1__run54
game1__run55
game1__run56
game1__run57
game1__run58
game1__run59
game1__run60
game1__run61
game1__run62
game1__run63
game1__run64
game1__run65
game1__run66
game1__run67
game1__run68
game1__run69
game1__run70
game1__run71
game1__run72
game1__run73
game1__run74
game1__run75
game1__run76
game1__ru

game1__run600
game1__run601
game1__run602
game1__run603
game1__run604
game1__run605
game1__run606
game1__run607
game1__run608
game1__run609
game1__run610
game1__run611
game1__run612
game1__run613
game1__run614
game1__run615
game1__run616
game1__run617
game1__run618
game1__run619
game1__run620
game1__run621
game1__run622
game1__run623
game1__run624
game1__run625
game1__run626
game1__run627
game1__run628
game1__run629
game1__run630
game1__run631
game1__run632
game1__run633
game1__run634
game1__run635
game1__run636
game1__run637
game1__run638
game1__run639
game1__run640
game1__run641
game1__run642
game1__run643
game1__run644
game1__run645
game1__run646
game1__run647
game1__run648
game1__run649
game1__run650
game1__run651
game1__run652
game1__run653
game1__run654
game1__run655
game1__run656
game1__run657
game1__run658
game1__run659
game1__run660
game1__run661
game1__run662
game1__run663
game1__run664
game1__run665
game1__run666
game1__run667
game1__run668
game1__run669
game1__run670
game1_

game2__run198
game2__run199
game2__run200
game2__run201
game2__run202
game2__run203
game2__run204
game2__run205
game2__run206
game2__run207
game2__run208
game2__run209
game2__run210
game2__run211
game2__run212
game2__run213
game2__run214
game2__run215
game2__run216
game2__run217
game2__run218
game2__run219
game2__run220
game2__run221
game2__run222
game2__run223
game2__run224
game2__run225
game2__run226
game2__run227
game2__run228
game2__run229
game2__run230
game2__run231
game2__run232
game2__run233
game2__run234
game2__run235
game2__run236
game2__run237
game2__run238
game2__run239
game2__run240
game2__run241
game2__run242
game2__run243
game2__run244
game2__run245
game2__run246
game2__run247
game2__run248
game2__run249
game2__run250
game2__run251
game2__run252
game2__run253
game2__run254
game2__run255
game2__run256
game2__run257
game2__run258
game2__run259
game2__run260
game2__run261
game2__run262
game2__run263
game2__run264
game2__run265
game2__run266
game2__run267
game2__run268
game2_

game2__run785
game2__run786
game2__run787
game2__run788
game2__run789
game2__run790
game2__run791
game2__run792
game2__run793
game2__run794
game2__run795
game2__run796
game2__run797
game2__run798
game2__run799
game2__run800
game2__run801
game2__run802
game2__run803
game2__run804
game2__run805
game2__run806
game2__run807
game2__run808
game2__run809
game2__run810
game2__run811
game2__run812
game2__run813
game2__run814
game2__run815
game2__run816
game2__run817
game2__run818
game2__run819
game2__run820
game2__run821
game2__run822
game2__run823
game2__run824
game2__run825
game2__run826
game2__run827
game2__run828
game2__run829
game2__run830
game2__run831
game2__run832
game2__run833
game2__run834
game2__run835
game2__run836
game2__run837
game2__run838
game2__run839
game2__run840
game2__run841
game2__run842
game2__run843
game2__run844
game2__run845
game2__run846
game2__run847
game2__run848
game2__run849
game2__run850
game2__run851
game2__run852
game2__run853
game2__run854
game2__run855
game2_

game3__run383
game3__run384
game3__run385
game3__run386
game3__run387
game3__run388
game3__run389
game3__run390
game3__run391
game3__run392
game3__run393
game3__run394
game3__run395
game3__run396
game3__run397
game3__run398
game3__run399
game3__run400
game3__run401
game3__run402
game3__run403
game3__run404
game3__run405
game3__run406
game3__run407
game3__run408
game3__run409
game3__run410
game3__run411
game3__run412
game3__run413
game3__run414
game3__run415
game3__run416
game3__run417
game3__run418
game3__run419
game3__run420
game3__run421
game3__run422
game3__run423
game3__run424
game3__run425
game3__run426
game3__run427
game3__run428
game3__run429
game3__run430
game3__run431
game3__run432
game3__run433
game3__run434
game3__run435
game3__run436
game3__run437
game3__run438
game3__run439
game3__run440
game3__run441
game3__run442
game3__run443
game3__run444
game3__run445
game3__run446
game3__run447
game3__run448
game3__run449
game3__run450
game3__run451
game3__run452
game3__run453
game3_

game3__run969
game3__run970
game3__run971
game3__run972
game3__run973
game3__run974
game3__run975
game3__run976
game3__run977
game3__run978
game3__run979
game3__run980
game3__run981
game3__run982
game3__run983
game3__run984
game3__run985
game3__run986
game3__run987
game3__run988
game3__run989
game3__run990
game3__run991
game3__run992
game3__run993
game3__run994
game3__run995
game3__run996
game3__run997
game3__run998
game3__run999
game4__run0
game4__run1
game4__run2
game4__run3
game4__run4
game4__run5
game4__run6
game4__run7
game4__run8
game4__run9
game4__run10
game4__run11
game4__run12
game4__run13
game4__run14
game4__run15
game4__run16
game4__run17
game4__run18
game4__run19
game4__run20
game4__run21
game4__run22
game4__run23
game4__run24
game4__run25
game4__run26
game4__run27
game4__run28
game4__run29
game4__run30
game4__run31
game4__run32
game4__run33
game4__run34
game4__run35
game4__run36
game4__run37
game4__run38
game4__run39
game4__run40
game4__run41
game4__run42
game4__run43
game

game4__run563
game4__run564
game4__run565
game4__run566
game4__run567
game4__run568
game4__run569
game4__run570
game4__run571
game4__run572
game4__run573
game4__run574
game4__run575
game4__run576
game4__run577
game4__run578
game4__run579
game4__run580
game4__run581
game4__run582
game4__run583
game4__run584
game4__run585
game4__run586
game4__run587
game4__run588
game4__run589
game4__run590
game4__run591
game4__run592
game4__run593
game4__run594
game4__run595
game4__run596
game4__run597
game4__run598
game4__run599
game4__run600
game4__run601
game4__run602
game4__run603
game4__run604
game4__run605
game4__run606
game4__run607
game4__run608
game4__run609
game4__run610
game4__run611
game4__run612
game4__run613
game4__run614
game4__run615
game4__run616
game4__run617
game4__run618
game4__run619
game4__run620
game4__run621
game4__run622
game4__run623
game4__run624
game4__run625
game4__run626
game4__run627
game4__run628
game4__run629
game4__run630
game4__run631
game4__run632
game4__run633
game4_

In [ ]:
agent_learning_row = {col_name: value for col_name, value in zip(X_column_names, X_values)}

In [18]:
agent_learning_row

{'X_0': 0,
 'X_1': 1,
 'X_2': 0,
 'X_3': 1,
 'X_4': 1,
 'X_5': 0,
 'X_6': 0,
 'X_7': 1,
 'X_8': 1,
 'X_9': 1}